In [ ]:
# hci4ai
# saving to disk 1000 jpeg heatmaps produced using gradcam
# the heatmaps correspond to the images in /images
# the only parameter for gradcam is "last_conv_layer_name" which is set to "dropout_1"

In [1]:
import re
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

###############################################################################

from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/marco_colab_data/hci4ai'

# load model
model_name = 'model__2023-04-26__22-42-58'
model = tf.keras.models.load_model(model_name)

###############################################################################

import matplotlib.cm as cm

def get_img_array(img_path, size):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=size)
    array = tf.keras.preprocessing.image.img_to_array(img)
    array = np.expand_dims(array, axis=0)
    return array

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):

    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    grads = tape.gradient(class_channel, last_conv_layer_output)

    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

###############################################################################

def save_gradcam(img_path, heatmap, alpha=0.4):

    img = tf.keras.preprocessing.image.load_img(img_path)
    img = tf.keras.preprocessing.image.img_to_array(img)

    heatmap = np.uint8(255 * heatmap)

    jet = cm.get_cmap("jet")

    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    jet_heatmap = tf.keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = tf.keras.preprocessing.image.img_to_array(jet_heatmap)

    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = tf.keras.preprocessing.image.array_to_img(superimposed_img)

    cam_path = "heatmap/h_" + img_path
    superimposed_img.save(cam_path)

###############################################################################

IMG_SIZE = (180, 180)

%cd '/content/drive/My Drive/marco_colab_data/hci4ai/export'

path = '/content/drive/My Drive/marco_colab_data/hci4ai/export'
files = (file for file in os.listdir(path) if os.path.isfile(os.path.join(path, file)))

last_conv_layer_name = "dropout_1"

for filename in files:
  img_array = get_img_array(filename, size=IMG_SIZE)
  heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)
  save_gradcam(filename, heatmap)

###############################################################################

drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')


Mounted at /content/drive
/content/drive/My Drive/marco_colab_data/hci4ai
/content/drive/My Drive/marco_colab_data/hci4ai/export


<ipython-input-1-ec5c0f92ad76>:61: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  jet = cm.get_cmap("jet")


All changes made in this colab session should now be visible in Drive.
